In [ ]:
import numpy as np
import pickle 
import copy
from sklearn.metrics.pairwise import cosine_similarity
import time

In [ ]:
filepath = 'fsd10_kbc.pkl'
f = open(filepath, 'rb')
fsd = pickle.load(f)
f.close()

In [ ]:
dataset = 'openpose_25'

if dataset == 'nturgb+d':
    pairs = [(0, 1), (1, 20), (2, 20), (3, 2), (4, 20), (5, 4), (6, 5), (7, 6), (8, 20), (9, 8),
                    (10, 9), (11, 10), (12, 0), (13, 12), (14, 13), (15, 14), (16, 0), (17, 16), (18, 17),
                    (19, 18), (21, 22), (20, 20), (22, 7), (23, 24), (24, 11)]
elif dataset == 'openpose': # 中心点在1点，为什么给了(0, 0)?后面改一改试一下
    pairs = ((0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (5, 1), (6, 5), (7, 6), (8, 2), (9, 8), (10, 9),
                    (11, 5), (12, 11), (13, 12), (14, 0), (15, 0), (16, 14), (17, 15))
elif dataset == 'coco':
    pairs = ((0, 0), (1, 0), (2, 0), (3, 1), (4, 2), (5, 0), (6, 0), (7, 5), (8, 6), (9, 7), (10, 8),
                    (11, 0), (12, 0), (13, 11), (14, 12), (15, 13), (16, 14))
elif dataset == 'openpose_25':
    pairs = ((0, 1), (1, 8), (2, 1), (3, 2), (4, 3), (5, 1), (6, 5), (7, 6), (8, 8), (9, 8), (10, 9), 
                    (11, 10), (12, 8), (13, 12), (14, 13), (15, 0), (16, 0), (17, 15), (18, 16), (19, 14), 
                    (20, 19), (21, 14), (22, 11), (23, 22), (24, 11))

In [ ]:
def JointToBone(results):

    keypoint = results['keypoint']
    M, T, V, C = keypoint.shape
    bone = np.zeros((M, T, V, C), dtype=np.float32)

    assert C in [2, 3]
    for v1, v2 in pairs:
        bone[..., v1, :] = keypoint[..., v1, :] - keypoint[..., v2, :]
        if C == 3 and dataset in ['openpose', 'coco', 'openpose_25']:
            score = (keypoint[..., v1, 2] + keypoint[..., v2, 2]) / 2
            bone[..., v1, 2] = score

    results['bone'] = bone
    return results

def BoneToCosine(results):
    #JointToBone(results)
    joint = results['keypoint']
    bone = results['bone']
    M, T, V, C = bone.shape
    cosine = np.ones((M, T, V, C), dtype=np.float32)*(-1)



    assert C == 3
    for v in range(V):# pairs长度与关节点长度(V个)相等，遍历每一个结点，计算关节余弦
        bones = []
        sign = []
        for v1, v2 in pairs: 
            if v1 == v:# 找到从i出的边
                bones.append(v1)
                sign.append('+')
            if v2 == v:# 找到往i出的边
                bones.append(v1)
                sign.append('-')
        if len(bones) >= 2:# 若该节点度大于2，即存在一个以上的cosine需要计算并求均值。以下为对原矩阵的拆解重建过程            
            for m in range(M):# M个样本 
                for t in range(T):# T个帧数
                    vectors = []
                    bone_scores = []
                    for b in range(len(bones)):
                        x,y,score = bone[m][t][bones[b]]
                        if sign[b] == '+':
                            vectors.append(np.array((+x, +y)))
                            #vectors.append(np.array((+x, +y)).reshape(1,-1))
                        if sign[b] == '-':
                            vectors.append(np.array((-x, -y)))
                            #vectors.append(np.array((-x, -y)).reshape(1,-1))
                        if C == 3 and dataset in ['openpose_25']:
                            bone_scores.append(score)
                    cosines = []
                    # sines = []
                    cos_scores = []
                    for v1 in range(len(vectors)):
                        for v2 in range(v1+1,len(vectors)):
                            if (np.linalg.norm(vectors[v1])!=0.0)&(np.linalg.norm(vectors[v2])!=0.0):
                                vec1_norm = vectors[v1] / np.linalg.norm(vectors[v1])
                                vec2_norm = vectors[v2] / np.linalg.norm(vectors[v2])
                                cos_sim = vec1_norm.dot(vec2_norm)
                            else:   
                                cos_sim = 0.0
                            # if abs(1 - cos_sim) < 1e-6:
                            #     sin_sim = 0.0
                            # else:
                            #     # 添加额外的检查以避免无效的开根号操作
                            #     sin_sim = np.sqrt(max(0, 1 - cos_sim**2))
                            cosines.append(cos_sim)
                            #sines.append(sin_sim)
                            if C == 3 and dataset in ['openpose_25']:
                                cos_scores.append((bone_scores[v1]+bone_scores[v2])/2)
                    cos = sum(cosines)/len(cosines)
                    #sin = sum(sines)/len(sines)
                    cosine[m][t][v][0] = cos # 赋值给第M个样本第T帧第i个结点
                    #cosine[m][t][v][1] = sin
                    if C == 3 and dataset in ['openpose_25']:# 计算置信度
                        cos_score = sum(cos_scores)/len(cos_scores)
                        cosine[m][t][v][2] = cos_score
        else:
            for m in range(M):# M个样本 
                for t in range(T):# T个帧数
                    vectors = []
                    bone_scores = []
                    x,y,score = bone[m][t][0]
                    if sign[0] == '+':
                        vectors.append(np.array((+x, +y)))
                        #vectors.append(np.array((+x, +y)).reshape(1,-1))
                    if sign[0] == '-':
                        vectors.append(np.array((-x, -y)))
                        #vectors.append(np.array((-x, -y)).reshape(1,-1))
                    if C == 3 and dataset in ['openpose_25']:
                        bone_scores.append(score)
                    vectors.append(np.array((-joint[m][t][v][0],-joint[m][t][v][1])))
                    bone_scores.append(joint[m][t][v][2])
                    cosines = []
                    cos_scores = []
                    v1 = 0
                    v2 = 1
                    if (np.linalg.norm(vectors[v1])!=0.0)&(np.linalg.norm(vectors[v2])!=0.0):
                        vec1_norm = vectors[v1] / np.linalg.norm(vectors[v1])
                        vec2_norm = vectors[v2] / np.linalg.norm(vectors[v2])
                        cos_sim = vec1_norm.dot(vec2_norm)
                    else:   
                        cos_sim = 0.0
                    cosines.append(cos_sim)
                    if C == 3 and dataset in ['openpose_25']:
                        cos_scores.append((bone_scores[v1]+bone_scores[v2])/2)
                    cos = sum(cosines)/len(cosines)
                    cosine[m][t][v][0] = cos # 赋值给第M个样本第T帧第i个结点
                    if C == 3 and dataset in ['openpose_25']:# 计算置信度
                        cos_score = sum(cos_scores)/len(cos_scores)
                        cosine[m][t][v][2] = cos_score
    results['cosine'] = cosine
    return results

def ToMotion(source,target,results):
    data = results[source]
    M, T, V, C = data.shape
    motion = np.zeros_like(data)

    assert C in [2, 3]
    motion[:, :T - 1] = np.diff(data, axis=1) #前T-1帧赋值为对帧数错位相减，第T帧不赋值，保留0矩阵
    if C == 3 and dataset in ['openpose', 'coco', 'openpose_25']:
        score = (data[:, :T - 1, :, 2] + data[:, 1:, :, 2]) / 2
        motion[:, :T - 1, :, 2] = score

    results[target] = motion

    return results

def gen_samplekey(results,key,name):
    data = results[key][...,-3:]
    M, T, V, C = data.shape
    data[...,-1][data[...,-1] == -1] = 0
    samplekey = np.zeros((M, T, V), dtype=np.float32)
    samplekey = data[:,:,:,0] * data[:,:,:,2]
    samplekey = np.sum(samplekey,axis=-1)
    results[name] = samplekey
    return results

def MergeSkeFeat(results):
    feats = []
    for name in ['bone']:
        feats.append(results.pop(name))
    feats = np.concatenate(feats, axis=-1)
    results['keypoint'] = feats
    return results
    
def GenSkeFeat(results):
    if 'keypoint_score' in results and 'keypoint' in results:
        assert dataset != 'nturgb+d'
        assert results['keypoint'].shape[-1] == 2, 'Only 2D keypoints have keypoint_score. '
        keypoint = results.pop('keypoint')
        keypoint_score = results.pop('keypoint_score')
        results['keypoint'] = np.concatenate([keypoint, keypoint_score[..., None]], -1)
    return results

# 不同的归一化策略
def Normalize1(results):
    for i in range(len(results)):
        x_max, x_min = np.max(results[i][...,0]), np.min(results[i][...,0])
        y_max, y_min = np.max(results[i][...,1]), np.min(results[i][...,1])
        results[i][..., 0] = (results[i][..., 0] - x_min) / (x_max - x_min)
        results[i][..., 1] = (results[i][..., 1] - y_min) / (y_max - y_min)
def Normalize2(results):
    for i in range(len(results)):
        x_max, x_min = np.max(results[i][...,0]), np.min(results[i][...,0])
        y_max, y_min = np.max(results[i][...,1]), np.min(results[i][...,1])
        results[i][..., 0] = (results[i][..., 0] - (x_max + x_min) / 2) / (x_max - x_min) * 2
        results[i][..., 1] = (results[i][..., 1] - (y_max + y_min) / 2) / (y_max - y_min) * 2

## draw the line of samplekey

In [ ]:
import matplotlib.pyplot as plt
def draw_samplekeys(samplekeys):
    plt.figure(figsize=(10,5))
    ax = plt.gca()
    ax.xaxis.set_ticks_position('top') # 将x轴的位置设置到顶部
    ax.xaxis.set_label_position('top') # 将x轴的标签设置到顶部
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    plt.plot(range(len(samplekeys[0])),samplekeys[0],marker='.')
    plt.xlabel('Frames')
    plt.ylabel('Indicators')
    plt.legend()
    # plt.savefig('keyframesampling.png',dpi=600)
    plt.show()

## data transform

In [ ]:
# for i in range(len(fsd['annotations'])):
#     fsd['annotations'][i]['joint'] = fsd['annotations'][i].pop('keypoint')
for i in range(len(fsd['annotations'])):
    start = time.time()
    data = fsd['annotations'][i]
    GenSkeFeat(data)
    BoneToCosine(data)
    # gen_samplekey(data, 'keypoint', 'samplekey')
    end = time.time()
    print('完成{}/{},耗时{:.2f}s.'.format(i,len(fsd['annotations']),end - start))

## motion transform

In [ ]:
for i in range(len(fsd['annotations'])):
    ToMotion('keypoint','joint_motion',fsd['annotations'][i])
    ToMotion('bone','bone_motion',fsd['annotations'][i])
    ToMotion('cosine','cosine_motion',fsd['annotations'][i])

## save

In [ ]:
with open('fsd10_kbc.pkl', 'wb') as f:
    pickle.dump(fsd, f)